# TP4 - Grupo 14

André Lucena Ribas Ferreira - A94956

Paulo André Alegre Pinto - A97391

# Enunciado do Problema 2

O programa `Python` seguinte implementa o algoritmo de bubble sort para ordenação in situ de um array de inteiros `seq`.

In [1]:
seq = [-2,1,2,-1,4,-4,-3,3]
changed = True
while changed:
    changed = False
    for i in range(len(seq) - 1):
        if seq[i] > seq[i+1]:
            seq[i], seq[i+1] = seq[i+1], seq[i]
            changed = True
print(seq)

[-4, -3, -2, -1, 1, 2, 3, 4]


 1. Defina a pré-condição e a pós-condição que descrevem a especificação deste algoritmo.
 2. O ciclo `for` pode ser descrito por uma transição $\,\mathtt{seq}\gets \mathit{exp}(\mathtt{seq})\,$. Construa uma relação de transição $\;\mathsf{trans}(\mathtt{seq},\mathtt{seq}')\,$ que modele esta atribuição.
 3. Usando a técnica que lhe parecer mais conveniente verifique a correção do algoritmo.

# Resolução

## 1. Pré-condição e pós-condição

As condições definidas para a resolução do problema são as demonstradas a seguir. No entanto, devido à implementação do algoritmo SAU demonstrado na teoria, não se coloca a redundância da terminação da execução do ciclo na pós-condição durante a implementação.

```python
    Pré-Condição: assume n >= 0 and changed == True
    Pós-Condução: assert forall j . 0 <= j < n-1 -> seq[j] <= seq[j+1] and changed == False
```

In [40]:
from pysmt.shortcuts import *
from pysmt.typing import *

De seguida, definem-se as variáveis usadas no problema.

In [41]:
N = 8
n = Int(N)
i = Symbol('i', INT)
seq = Symbol('seq', ArrayType(INT, INT))
f = Symbol('f', FunctionType(INT,[INT]))

A função `prove(g)` recebe um predicado e testa se é possível prová-lo para todo o traço de execução. Esta função é utilizada como auxiliar para testar a funcionalidade das implementações.

In [42]:
def prove(g):
    with Solver(name="z3") as s:
        s.add_assertion(Not(g))
        if s.solve():
            print("Failed to prove.")
            for inc in range(N):
                print(s.get_value(Select(seq, Int(inc))))
            for inc in range(N):
                x = f(Int(inc))
                print("f", s.get_value(x))
                #print("f", s.get_value())
        else:
            print("Proved.")

As seguintes funções são funções auxiliares à manipulação das variáveis da execução por parte do algoritmo SAU.

In [43]:
def prime(v):
    return Symbol("next(%s)" % v.symbol_name(), v.symbol_type())
def fresh(v):
    return FreshSymbol(typename=v.symbol_type(),template=v.symbol_name()+"_%d")

## 2. Definição da transição

### Transição com passos intermédios

A seguinte função `trans_seq`, auxiliada pela função `aux`, serve para realizar a transição entre $seq$ e $seq´$. Para tal, criam-se *Arrays* intermédios onde se executam as atribuições, par a par de valores.

In [51]:
def aux(seq,seqlinha,I):
    i = Int(I)
    l = list()
    """for n in range(N):
        if n != I and n != I+1:
            l.append(Equals(Select(seqlinha,Int(n)), Select(seq,Int(n))))# copia o seq para o seqlinha
    
    l.append(Equals(Select(seqlinha, i), Ite(Select(seq,i) < Select(seq, i+Int(1)), Select(seq,i), Select(seq, i+Int(1)))))
    l.append(Equals(Select(seqlinha, i+Int(1)), Ite(Select(seq,i) > Select(seq, i+Int(1)), Select(seq,i), Select(seq, i+Int(1)))))
    
    for form in l:
        print(form.serialize())
        
    print()"""
    return And(l)
    
def trans_seq(seq,seq_p,changed,changed_p):
    seqlist = []
    seqlist.append(seq)
    for i in range(1, N-1):
        seqlist.append(Symbol('seq_i' + str(i), ArrayType(INT,INT)))
    seqlist.append(seq_p)
    
    troca = Iff(Iff(changed_p, Bool(False)), (Equals(seq,seq_p)))
        
    return And(And([aux(seqlist[i],seqlist[i+1],i) for i in range(N-1)]),troca)

In [52]:
def test_solver(conditions):
    with Solver(name="z3") as s:
        if s.solve(conditions):
            for inc in range(N):
                print(s.get_value(Select(seq, Int(inc))))
            print()
            for inc in range(N):
                print(s.get_value(Select(prime(seq), Int(inc))))
        else:
            print("Unsat")

In [53]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), store1)  
test_solver([pre, trans_seq(seq,prime(seq),changed,prime(changed))])

-2
1
2
-1
4
-4
-3
3

10
10
10
10
10
10
10
10


### Transição através de função recursiva

A função `for_cycle` serve como função de transição tendo em consideração a atribuição a cada uma das entradas do array. Para cada entrada $0 \leq i<n-1$ `seq[i]` atribui-se o menor dos valores entre `seq[i+1]` e o máximo valor de todos os `seq[j]`, para $0 \leq j<i$.

Para tal 

In [10]:
ax1 = Equals(f(Int(0)), Select(seq, Int(0)))
ax2 = ForAll([i], Implies(And(i>Int(0), i<n), Equals(f(i), Ite(Select(seq,i) >= f(i-Int(1)), Select(seq,i), f(i-Int(1))))))
axioms = And(ax1,ax2)

In [11]:
store1 = And(Equals(Select(seq, Int(0)), Int(-2) ),
            Equals(Select(seq, Int(1)), Int(1)  ),
            Equals(Select(seq, Int(2)), Int(2)  ),
            Equals(Select(seq, Int(3)), Int(-1) ),
            Equals(Select(seq, Int(4)), Int(4)  ),
            Equals(Select(seq, Int(5)), Int(-4) ),
            Equals(Select(seq, Int(6)), Int(-3) ),
            Equals(Select(seq, Int(7)), Int(3)  ))

prove(Implies(And(axioms, store1), Equals(f(Int(1)), Int(1))))
prove(Implies(And(axioms, store1), Equals(f(Int(5)), Int(4))))
prove(Implies(And(axioms, store1), Equals(f(Int(5)), Int(1))))

Proved.
Proved.
Failed to prove.
-2
1
2
-1
4
-4
-3
3
f -2
f 1
f 2
f 2
f 4
f 4
f 4
f 4


In [18]:
def for_cycle(c, p, changed_p):
    indutivo = ForAll([i], Implies(And(i<n-Int(1), i>=Int(0)), 
                                   And(Equals(Select(p,i), Ite(Select(c, i+Int(1)) <= f(i), Select(c, i+Int(1)), f(i))))))
    
    final = Equals(Select(p, n - Int(1)), f(n - Int(1)))
    
    troca = Iff(Iff(changed, Bool(True)), Not(Equals(c,p)))
    
    return And(indutivo, troca, final)

In [26]:

# A classe "Sigle Assignment Unfold"
class SAU(object):
    """Trivial representation of a while cycle and its unfolding."""
    def __init__(self, variables, pre , pos, control, trans, sname="z3"):
              
        self.variables = variables       # variables   
        self.pre = pre                   # pre-condition as a predicate in "variables"
        self.pos = pos                   # pos-condition as a predicate in "variables"
        self.control = control           # cycle control as a predicate in "variables"
        self.trans = trans               # cycle body as a binary transition relation 
                                         # in "variables" and "prime variables"
        
        self.prime_variables = [prime(v) for v in self.variables]
        self.frames = [And([Not(control),pos])]  
                 # inializa com uma só frame: a da terminação do ciclo
        
        self.solver = Solver(name=sname)
        self.fresh = []

    def new_frame(self):        
        freshs = [fresh(v) for v in self.variables]    
        b = self.control
        S = self.trans.substitute(dict(zip(self.prime_variables,freshs)))
        W = self.frames[-1].substitute(dict(zip(self.variables,freshs)))
        self.frames.append(And([b , ForAll(freshs, Implies(S, W))]))
        self.fresh += freshs
        
    def unfold(self,bound=0):
        n = 0
        while True:
            if n > bound:
                print("falha: número de tentativas ultrapassa o limite %d "%bound)
                break
            
            f = Or(self.frames)               
            if self.solver.solve([self.pre,Not(f)]):
                """print(self.solver.get_value(changed))
                print("seq:")
                for inc in range(N):
                    print(self.solver.get_value(Select(seq, Int(inc))))
                print("freshs:")
                for v in self.fresh:
                    if str(v.symbol_type()) == "Array{Int, Int}":
                        for inc in range(N):
                            print(v.symbol_name(), inc, ":", self.solver.get_value(Select(v, Int(inc))))
                    else:
                        print(v.symbol_name(),":", self.solver.get_value(v))
                print("end frame")"""
                self.new_frame()
                n += 1
            else:
                print("sucesso na tentativa %d "%n)
                break   

In [27]:
changed = Symbol('changed', BOOL)
variables = [changed, seq]

In [29]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms, store1)
t = for_cycle(seq,prime(seq),prime(changed))
test_solver([t, pre])

-2
1
2
-1
4
-4
-3
3

-2
1
-1
2
-4
-3
3
4


In [30]:
test_solver([Not(t), pre])

-2
1
2
-1
4
-4
-3
3

8
8
8
8
8
8
8
8


In [31]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms, store1)  
pos  =  And(ForAll([i], Implies(And(i>=0, i<n-1), Select(seq, i) <= Select(seq, i+1))))
cond =  Iff(changed, Bool(True))               # condição de controlo do ciclo
trans = for_cycle(seq, prime(seq), prime(changed))

#, 
#Iff(Iff(prime(changed), Bool(False)),
#ForAll([i], And(i>=Int(0), i<n, Equals(Select(seq, i), Select(prime(seq), i))))))

W = SAU(variables, pre, pos, cond, trans)
W.unfold(8)

falha: número de tentativas ultrapassa o limite 8 


In [ ]:
store2 = And([Equals(Select(seq, Int(i)), Int(-2)) for i in range(N-1)] + [Equals(Select(seq, Int(7)), Int(-3))])
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms, store2)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

((seq[0] = -2) & (seq[1] = -2) & (seq[2] = -2) & (seq[3] = -2) & (seq[4] = -2) & (seq[5] = -2) & (seq[6] = -2) & (seq[7] = -3))
falha: número de tentativas ultrapassa o limite 9 


In [33]:
store3 = And([Equals(Select(seq, Int(i)), Int(-2)) for i in range(N)])
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms, store3)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

((seq[0] = -2) & (seq[1] = -2) & (seq[2] = -2) & (seq[3] = -2) & (seq[4] = -2) & (seq[5] = -2) & (seq[6] = -2) & (seq[7] = -2))
falha: número de tentativas ultrapassa o limite 9 


In [35]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), axioms)
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

falha: número de tentativas ultrapassa o limite 9 


(seq_i1[2] = seq[2])
(seq_i1[3] = seq[3])
(seq_i1[4] = seq[4])
(seq_i1[5] = seq[5])
(seq_i1[6] = seq[6])
(seq_i1[7] = seq[7])
(seq_i1[0] = ((seq[0] < seq[(0 + 1)]) ? seq[0] : seq[(0 + 1)]))
(seq_i1[(0 + 1)] = ((seq[(0 + 1)] < seq[0]) ? seq[0] : seq[(0 + 1)]))

(seq_i2[0] = seq_i1[0])
(seq_i2[3] = seq_i1[3])
(seq_i2[4] = seq_i1[4])
(seq_i2[5] = seq_i1[5])
(seq_i2[6] = seq_i1[6])
(seq_i2[7] = seq_i1[7])
(seq_i2[1] = ((seq_i1[1] < seq_i1[(1 + 1)]) ? seq_i1[1] : seq_i1[(1 + 1)]))
(seq_i2[(1 + 1)] = ((seq_i1[(1 + 1)] < seq_i1[1]) ? seq_i1[1] : seq_i1[(1 + 1)]))

(seq_i3[0] = seq_i2[0])
(seq_i3[1] = seq_i2[1])
(seq_i3[4] = seq_i2[4])
(seq_i3[5] = seq_i2[5])
(seq_i3[6] = seq_i2[6])
(seq_i3[7] = seq_i2[7])
(seq_i3[2] = ((seq_i2[2] < seq_i2[(2 + 1)]) ? seq_i2[2] : seq_i2[(2 + 1)]))
(seq_i3[(2 + 1)] = ((seq_i2[(2 + 1)] < seq_i2[2]) ? seq_i2[2] : seq_i2[(2 + 1)]))

(seq_i4[0] = seq_i3[0])
(seq_i4[1] = seq_i3[1])
(seq_i4[2] = seq_i3[2])
(seq_i4[5] = seq_i3[5])
(seq_i4[6] = seq_i3[6])
(seq_i4[7] = 

In [38]:
trans = trans_seq(seq,prime(seq),changed,prime(changed))
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), store1)  
W = SAU(variables, pre, pos, cond, trans)
W.unfold(8)

(seq_i1[2] = seq[2])
(seq_i1[3] = seq[3])
(seq_i1[4] = seq[4])
(seq_i1[5] = seq[5])
(seq_i1[6] = seq[6])
(seq_i1[7] = seq[7])
(seq_i1[0] = ((seq[0] < seq[(0 + 1)]) ? seq[0] : seq[(0 + 1)]))
(seq_i1[(0 + 1)] = ((seq[(0 + 1)] < seq[0]) ? seq[0] : seq[(0 + 1)]))

(seq_i2[0] = seq_i1[0])
(seq_i2[3] = seq_i1[3])
(seq_i2[4] = seq_i1[4])
(seq_i2[5] = seq_i1[5])
(seq_i2[6] = seq_i1[6])
(seq_i2[7] = seq_i1[7])
(seq_i2[1] = ((seq_i1[1] < seq_i1[(1 + 1)]) ? seq_i1[1] : seq_i1[(1 + 1)]))
(seq_i2[(1 + 1)] = ((seq_i1[(1 + 1)] < seq_i1[1]) ? seq_i1[1] : seq_i1[(1 + 1)]))

(seq_i3[0] = seq_i2[0])
(seq_i3[1] = seq_i2[1])
(seq_i3[4] = seq_i2[4])
(seq_i3[5] = seq_i2[5])
(seq_i3[6] = seq_i2[6])
(seq_i3[7] = seq_i2[7])
(seq_i3[2] = ((seq_i2[2] < seq_i2[(2 + 1)]) ? seq_i2[2] : seq_i2[(2 + 1)]))
(seq_i3[(2 + 1)] = ((seq_i2[(2 + 1)] < seq_i2[2]) ? seq_i2[2] : seq_i2[(2 + 1)]))

(seq_i4[0] = seq_i3[0])
(seq_i4[1] = seq_i3[1])
(seq_i4[2] = seq_i3[2])
(seq_i4[5] = seq_i3[5])
(seq_i4[6] = seq_i3[6])
(seq_i4[7] = 

In [39]:
pre  =  And(n>=Int(0), Iff(changed, Bool(True)), store2)  
W = SAU(variables, pre, pos, cond, trans)
W.unfold(9)

sucesso na tentativa 2 
